# 🚀 Pipeline de Churn Prediction - Arquitetura Medallion

Este notebook simula a transição da camada **Silver** para a **Gold**, aplicando engenharia de atributos (Feature Engineering) e treinando um modelo de Machine Learning para prever o cancelamento de clientes (Churn). Foi otimizado para a infraestrutura do Databricks Free Edition.

## 1. Instalação e Importação de Bibliotecas
O Databricks já possui o PySpark nativamente, mas vamos importar as funções necessárias para o nosso pipeline de ML.

In [ ]:
from pyspark.sql.functions import col, datediff, current_date, to_date
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

## 2. Ingestão de Dados (Camada Silver)
Nesta etapa, carregamos os dados já limpos e estruturados da nossa tabela (ou arquivo CSV) na camada Silver.

In [ ]:
# Lendo o CSV mapeado no Unity Catalog / Volumes (Camada Silver)
file_path = "/Volumes/workspace/voc/churn/churn_silver_2025.csv"

df_silver = spark.read.csv(file_path, header=True, inferSchema=True)
display(df_silver.limit(5))

## 3. Tratamento e Feature Engineering (Silver ➡️ Gold)
Criaremos novas variáveis (features) que ajudarão o algoritmo a encontrar padrões.
* **taxa_uso_valor**: Relação entre os logs de uso e a mensalidade paga.
* **dias_desde_assinatura**: O tempo de vida (tenure) do cliente na base.

In [ ]:
df_gold_prep = df_silver.withColumn(
    "taxa_uso_valor", 
    col("total_logs_app_30d") / (col("valor_mensalidade") + 0.01) # Evita divisão por zero
).withColumn(
    "dias_desde_assinatura", 
    datediff(current_date(), to_date(col("data_assinatura")))
).na.drop() # Tratamento básico de eventuais nulos

display(df_gold_prep.limit(5))

## 4. Preparação para Machine Learning
Modelos precisam de números para operar. Aqui convertemos textos para índices e agrupamos todas as variáveis num vetor de características (`features`).

In [ ]:
# Tratamento da Variável Categórica
indexer = StringIndexer(inputCol="categoria_principal_voc", outputCol="categoria_indexada", handleInvalid="keep")
df_indexed = indexer.fit(df_gold_prep).transform(df_gold_prep)

# Vetorização das Features
assembler = VectorAssembler(
    inputCols=["valor_mensalidade", "total_logs_app_30d", "tickets_suporte_abertos", 
               "score_sentimento_voc", "taxa_uso_valor", "dias_desde_assinatura", "categoria_indexada"],
    outputCol="features"
)
df_features = assembler.transform(df_indexed)

## 5. Divisão de Dados e Treinamento do Modelo
Separamos **80% dos dados para treino** e **20% para teste**. Em seguida, instanciamos o `RandomForestClassifier`.

In [ ]:
# Split de Treino e Teste
train_data, test_data = df_features.randomSplit([0.8, 0.2], seed=42)

# Treinamento do Random Forest
# Parâmetros enxutos (numTrees=50, maxDepth=5) para rodar suavemente na Free Edition
rf = RandomForestClassifier(featuresCol="features", labelCol="churn", numTrees=50, maxDepth=5, seed=42)
rf_model = rf.fit(train_data)

## 6. Avaliação do Modelo
Testamos a qualidade da nossa IA em dados não vistos previamente utilizando duas métricas de classificação robustas.

In [ ]:
predictions = rf_model.transform(test_data)

# Cálculo da AUC-ROC
evaluator_roc = BinaryClassificationEvaluator(labelCol="churn", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc_roc = evaluator_roc.evaluate(predictions)

# Cálculo da Acurácia
evaluator_acc = MulticlassClassificationEvaluator(labelCol="churn", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_acc.evaluate(predictions)

print("=" * 30)
print(f"📊 Acurácia do Modelo: {accuracy:.4f}")
print(f"📈 AUC-ROC: {auc_roc:.4f}")
print("=" * 30)

## 7. Geração da Tabela Gold e Conclusão
A base final `churn_predictions_gold` consolida o `id_cliente`, as probabilidades geradas pelas árvores de decisão e a classificação binária final.

In [ ]:
churn_predictions_gold = predictions.select("id_cliente", "probability", "prediction") \
                                    .withColumnRenamed("prediction", "previsao_churn")

# Visualizando o DataFrame Gold
display(churn_predictions_gold)

# Em um ambiente produtivo real, o comando abaixo salvaria na camada Gold:
# churn_predictions_gold.write.mode("overwrite").saveAsTable("gold.churn_predictions")

### ✅ Conclusão
O pipeline foi executado com sucesso! 
Os dados brutos (Silver) foram higienizados e transformados matematicamente (Feature Eng). Após serem indexados e vetorizados, passaram por um classificador de Floresta Aleatória que identificou o padrão de evasão. Agora possuímos um Dataframe na camada Gold enriquecido com a probabilidade de evasão, pronto para ser plugado em painéis ou fluxos de CRM de retenção.